In [21]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import re
from fastai.text import *

pd.set_option('display.max_colwidth', 500)



In [22]:
df09 = pd.read_json('condensed_2009.json')
df10 = pd.read_json('condensed_2010.json')
df11 = pd.read_json('condensed_2011.json')
df12 = pd.read_json('condensed_2012.json')
df13 = pd.read_json('condensed_2013.json')
df14 = pd.read_json('condensed_2014.json')
df15 = pd.read_json('condensed_2015.json')
df16 = pd.read_json('condensed_2016.json')
df17 = pd.read_json('condensed_2017.json')
df18 = pd.read_json('condensed_2018.json')
df19 = pd.read_json('2019.json')
df20 = pd.read_json('2020.json')

years = [df09, df10, df11, df12, df13, df14, 
         df14, df15, df16, df17, df18, df19, df20]

df = pd.concat(years)
df = df.sort_values('created_at').reset_index(drop=True)

df['created_date'] = [d.date() for d in df['created_at']]
df['created_time'] = [d.time() for d in df['created_at']]

# takes a retweet as a string and removes the 'RT @name: '
def remove_rt_signature(tweet):
    split_tweet = tweet.split(':', 1)
    return split_tweet[1][1:]

# make a new column of all the tweets with the retweets cleaned up
df['text_cleaned'] = df.apply(lambda row: row['text'] if row['is_retweet'] == False 
                                    else remove_rt_signature(row['text']), axis=1)



# make a column that indicates the hour of tweet

df['created_hour'] = pd.to_datetime(df['created_time'].astype(str)).dt.hour



# make a column that classifies tweet as night or day tweet (2:00 - 10:00 GMT = 22:00 - 6:00 EST)

df['is_night_tweet'] = df['created_hour'].isin([2, 3, 4, 5, 6, 7, 8, 9, 10])


# make a column that classifies tweets as before and after becoming president

df['is_presidential'] = False 


# add a validation column to indicate the validation data, eventually set most recent 10-20% to True

df['is_valid'] = False



In [24]:
df.to_csv('tweet_df.csv')

In [ ]:
# Validation data will be most recent 10% of the tweets

# test_size = len(df['is_valid']) // 10 

# df['is_valid'].iloc[-test_size:] = True    

In [26]:
df.head()

,source,id_str,text,created_at,retweet_count,in_reply_to_user_id_str,favorite_count,is_retweet,created_date,created_time,text_cleaned,created_hour,is_night_tweet,is_valid
0,Twitter Web Client,1698308935,Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!,2009-05-04 18:54:25+00:00,253,NaN,202,False,2009-05-04,18:54:25,Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!,18,False,False
1,Twitter Web Client,1701461182,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!,2009-05-05 01:00:10+00:00,2,NaN,3,False,2009-05-05,01:00:10,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!,1,False,False
2,Twitter Web Client,1737479987,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!,2009-05-08 13:38:08+00:00,3,NaN,2,False,2009-05-08,13:38:08,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!,13,False,False
3,Twitter Web Client,1741160716,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e,2009-05-08 20:40:15+00:00,8,NaN,27,False,2009-05-08,20:40:15,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e,20,False,False
4,Twitter Web Client,1773561338,"""My persona will never be that of a wallflower - I’d rather build walls than cling to them"" --Donald J. Trump",2009-05-12 14:07:28+00:00,1421,NaN,1950,False,2009-05-12,14:07:28,"""My persona will never be that of a wallflower - I’d rather build walls than cling to them"" --Donald J. Trump",14,False,False


In [27]:
df[df.is_night_tweet == True].shape

(11384, 14)

In [28]:
df.shape

(53228, 14)